In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
countries = ['Finland', 'USA', 'Japan', 'Thailand', 'United Kingdom', 'Germany',
       'China', 'Hong Kong (SAR) China', 'Sweden', 'Singapore', 'Spain',
       'Korea Republic of', 'France', 'Denmark', 'India',
       'Russian Federation', 'Italy', 'Norway', 'Greece', 'Netherlands',
       'Estonia', 'Poland', 'Switzerland', 'Hungary', 'Austria',
       'Czech Republic', 'Portugal']

nationalities = ['FI', 'SE', 'DE', 'JP', 'US', 'GB', 'CN', 'RU', 'KR', 'HK', 'NO',
       'AU', 'FR', 'EE', 'DK']

## Data preprocesing automotization

In [420]:
def preprocess_data(df, nationalities, countries):
#     df['target'] = df['SSR_CODE'] == 'VGML'
    df['target'] = ~df['SSR_CODE'].isna()
    df = df.drop(columns=['DV_LEG_H_ID','DV_SEGMENT_H_ID', 'TICKET_TYPE','ALDES','DEPSTN', 'AIRCRAFT_REGISTRATION', 'AIRCRAFT_SUBTYPE',  'FLTDATE_LOCAL', 'NATIONALITY', 'SSR_CODE', 'SSR_SEAT_MEAL'])
    df['LDEPTIME_LOCAL'] = pd.to_datetime(df['LDEPTIME_LOCAL'])
    
    df = df[df.GENDER2.isin(['Male', 'Female'])]
    df['GENDER2'] = df['GENDER2'] == 'Male'
    
    df = df[df.POINT_OF_SALE.isin(nationalities) & (df.ARR_COUNTRY.isin(countries))]
    df = df.drop(columns=['POINT_OF_SALE'])
    
    df = df.drop(columns=['ARRSTN'])
    
    df = df[df.SERVICE_CLASS.isin(['ECONOMY', 'BUSINESS'])]
    df['SERVICE_CLASS'] = df['SERVICE_CLASS'] == 'BUSINESS'
    
    df['FLIGHT_DURATION'] = df['FLIGHT_DURATION'] / 60
    
    df['ROUTE_TYPE'] = df['ROUTE_TYPE'] == 'LH'
    
    df['IS_DAYTIME'] = df['LDEPTIME_LOCAL'].apply(lambda x: x.hour >= 6 and x.hour <= 18)
    df['SEASON'] = df['LDEPTIME_LOCAL'].apply(lambda x: (x.month % 12) // 3)
    
    df = df.drop(columns=['FLTNBR', 'LDEPTIME_LOCAL'])
    
    df = pd.get_dummies(df, columns=['ARR_COUNTRY', 'SEASON','TRANSFER_STATUS','BOOKING_CLASS'])
    
    labels = df[['target']].astype('int32')
    df = df.drop(columns=['target'])
    
    return df, labels

In [421]:
train_data = pd.read_csv('ALL1.csv').sample(100000)
train_preprocessed_data, train_labels = preprocess_data(train_data, nationalities, countries)
# train_preprocessed_data = train_preprocessed_data[['FLIGHT_DURATION', 'ROUTE_TYPE', 'IS_FPLUS', 'GENDER2', 'SERVICE_CLASS']]

C:\Users\AY54025\Anaconda3\envs\project\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [422]:
test_data = pd.read_csv('ALL1.csv').sample(10000)
test_preprocessed_data, test_labels = preprocess_data(test_data, nationalities, countries)
# test_preprocessed_data = test_preprocessed_data[['FLIGHT_DURATION', 'ROUTE_TYPE', 'IS_FPLUS', 'GENDER2', 'SERVICE_CLASS']]

## model 1 training 

In [423]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

model_1 = DecisionTreeClassifier()

In [424]:
model_1.fit(train_preprocessed_data.values, train_labels.values)

DecisionTreeClassifier()

In [425]:
y_test =  np.squeeze(train_labels.values)
threshold = 0.05

y_pred = np.array(model_1.predict_proba(train_preprocessed_data.values)[:, 1])
y_pred  = y_pred > threshold
y_pred = y_pred.astype(int)  

print(f"XGB accuracy on the test set : {round(np.sum(y_pred == y_test) / len(y_pred), 4)}")

XGB accuracy on the test set : 0.9468


In [393]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print ('Accuracy:', accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred))
print ('Recall:', recall_score(y_test, y_pred))
print ('Precision:', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))

Accuracy: 0.9436843425195753
F1 score: 0.5287945908137095
Recall: 0.9969230769230769
Precision: 0.3598286530223703

 clasification report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97     69499
           1       0.36      1.00      0.53      2275

    accuracy                           0.94     71774
   macro avg       0.68      0.97      0.75     71774
weighted avg       0.98      0.94      0.96     71774


 confussion matrix:
 [[65464  4035]
 [    7  2268]]


In [439]:
feature_importances = pd.DataFrame(model_1.feature_importances_,
                                   index = test_preprocessed_data.columns,
                                    columns=['importances']).sort_values('importances', ascending=False)
display(feature_importances.head(7))

,importances
FLIGHT_DURATION,0.281487
BOOKING_CLASS_A,0.137700
GENDER2,0.054873
SEASON_3,0.050707
SEASON_2,0.040417
SEASON_1,0.040082
SEASON_0,0.036106


## model 2 training

In [394]:
index = [bool(p) for p in y_pred]
train2_preprocessed_data, train2_labels = train_preprocessed_data[index], train_labels[index]

In [395]:
model_2 = DecisionTreeClassifier()

In [396]:
model_2.fit(train2_preprocessed_data.values, train2_labels.values)

DecisionTreeClassifier()

In [407]:
y_test =  np.squeeze(train2_labels.values)
threshold_2 = 0.4

y_pred = np.array(model_2.predict_proba(train2_preprocessed_data.values)[:, 1])
y_pred  = y_pred > threshold_2
y_pred = y_pred.astype(int)  

print(f"XGB accuracy on the test set : {round(np.sum(y_pred == y_test) / len(y_pred), 4)}")

XGB accuracy on the test set : 0.772


In [409]:
print ('Accuracy:', accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred))
print ('Recall:', recall_score(y_test, y_pred))
print ('Precision:', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))

Accuracy: 0.772013326987149
F1 score: 0.6701859077346799
Recall: 0.6437389770723104
Precision: 0.6988989947343226

 clasification report:
               precision    recall  f1-score   support

           0       0.81      0.84      0.83      4035
           1       0.70      0.64      0.67      2268

    accuracy                           0.77      6303
   macro avg       0.75      0.74      0.75      6303
weighted avg       0.77      0.77      0.77      6303


 confussion matrix:
 [[3406  629]
 [ 808 1460]]


In [440]:
feature_importances = pd.DataFrame(model_2.feature_importances_,
                                   index = train2_preprocessed_data.columns,
                                    columns=['importances']).sort_values('importances', ascending=False)
display(feature_importances.head(7))

,importances
FLIGHT_DURATION,0.239309
BOOKING_CLASS_A,0.082696
IS_FPLUS,0.078191
TRANSFER_STATUS_has_transferred,0.064150
GENDER2,0.039214
ARR_COUNTRY_Italy,0.031460
SERVICE_CLASS,0.029517


## Model evaluation

In [410]:
val_data = pd.read_csv('ALL1.csv').sample(10000)
val_preprocessed_data, val_labels = preprocess_data(val_data, nationalities, countries)

C:\Users\AY54025\Anaconda3\envs\project\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [411]:
y_pred = np.array(model_1.predict_proba(val_preprocessed_data.values)[:, 1])
y_pred = y_pred > threshold
y_pred = y_pred.astype(int)  

In [412]:
index = [bool(p) for p in y_pred]
val2_preprocessed_data, val2_labels = val_preprocessed_data[index], val_labels[index]

In [413]:
y_pred_2 = np.array(model_2.predict_proba(val2_preprocessed_data.values)[:, 1])
y_pred_2 = y_pred_2 > threshold_2
y_pred_2 = y_pred_2.astype(int)  

In [414]:
pred_labels = y_pred
pred_labels[pred_labels == 1] = y_pred_2

In [415]:
y_test = np.squeeze(val_labels.values)

In [416]:
print ('Accuracy:', accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred))
print ('Recall:', recall_score(y_test, y_pred))
print ('Precision:', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))

Accuracy: 0.9586159169550174
F1 score: 0.3126436781609196
Recall: 0.31627906976744186
Precision: 0.3090909090909091

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98      7010
           1       0.31      0.32      0.31       215

    accuracy                           0.96      7225
   macro avg       0.64      0.65      0.65      7225
weighted avg       0.96      0.96      0.96      7225


 confussion matrix:
 [[6858  152]
 [ 147   68]]


## ------------------------
## Outlier Detection

In [ ]:
train_data = pd.read_csv('ALL1.csv').sample(50000)
train_preprocessed_data, train_labels = preprocess_data(train_data, nationalities, countries)
train_preprocessed_data = train_preprocessed_data[['FLIGHT_DURATION', 'ROUTE_TYPE', 'IS_FPLUS', 'GENDER2', 'SERVICE_CLASS']]

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix

clf = IsolationForest(contamination = float(np.sum(train_labels) / train_labels.shape[0])).fit(train_preprocessed_data)

In [ ]:
test_data = pd.read_csv('ALL1.csv').sample(10000)
test_preprocessed_data, test_labels = preprocess_data(test_data, nationalities, countries)
test_preprocessed_data = test_preprocessed_data[['FLIGHT_DURATION', 'ROUTE_TYPE', 'IS_FPLUS', 'GENDER2', 'SERVICE_CLASS']]

In [ ]:
y_pred = clf.predict(test_preprocessed_data)

In [ ]:
y_pred[y_pred == 1] = 0
y_pred[y_pred == -1] = 1

print ('Confussion matrix:\n', confusion_matrix(test_labels, y_pred))

In [ ]:
lof = LocalOutlierFactor(novelty=True)
lof.fit(train_preprocessed_data)

In [ ]:
y_pred = lof.predict(test_preprocessed_data)

In [ ]:
y_pred[y_pred == 1] = 0
y_pred[y_pred == -1] = 1

print ('Confussion matrix:\n', confusion_matrix(test_labels, y_pred))